In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("all_crypto_data.csv")

In [4]:
display(data)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,24364.000000,4.745209e+11,1.0,5.119166e+11,1.624614e+10,24433.000000,...,2.100000e+07,2.100000e+07,59717.00,-59.18370,2021-11-10T14:24:11.849Z,51.300000,47414.53965,2013-07-05T00:00:00.000Z,NaN,2023-08-23T18:42:47.046Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1546.270000,1.859876e+11,2.0,1.859876e+11,1.187268e+10,1553.380000,...,1.202143e+08,NaN,4228.93,-63.42011,2021-12-01T08:38:24.623Z,0.381455,405435.72792,2015-10-20T00:00:00.000Z,"{'times': 83.84405195701511, 'currency': 'btc'...",2023-08-23T18:42:50.617Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.920031,7.622065e+10,3.0,7.622065e+10,1.414680e+10,0.926721,...,8.284578e+10,NaN,1.13,-18.69647,2018-07-24T00:00:00.000Z,0.533096,72.58266,2015-03-02T00:00:00.000Z,NaN,2023-08-23T18:40:00.329Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,199.990000,3.077853e+10,4.0,4.000949e+10,6.612592e+08,200.630000,...,1.538562e+08,2.000000e+08,583.17,-65.70086,2021-11-26T02:58:28.000Z,0.033599,595218.98232,2017-10-19T00:00:00.000Z,NaN,2023-08-23T18:42:50.796Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.486917,2.577040e+10,5.0,4.870225e+10,9.332927e+08,0.491048,...,9.998849e+10,1.000000e+11,2.82,-82.74995,2018-01-07T00:00:00.000Z,0.001946,24921.86548,2013-08-16T00:00:00.000Z,NaN,2023-08-23T18:42:47.909Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10145,cola-token-2,cola,Cola Token,https://assets.coingecko.com/coins/images/3093...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.00000,NaN,NaN,NaN
10146,fiero,fiero,Fiero,https://assets.coingecko.com/coins/images/3036...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.00000,NaN,NaN,NaN
10147,amplifi-dao,agg,AmpliFi DAO,https://assets.coingecko.com/coins/images/2913...,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000e+09,NaN,NaN,0.00000,NaN,NaN,0.00000,NaN,NaN,NaN
10148,ayin,ayin,Ayin,https://assets.coingecko.com/coins/images/3131...,NaN,NaN,NaN,NaN,NaN,NaN,...,1.924590e+05,4.000000e+06,NaN,0.00000,NaN,NaN,0.00000,NaN,NaN,NaN


In [10]:
import requests
import pandas as pd
from datetime import datetime

# Function to fetch historical data
def fetch_historical_data(coin_id, vs_currency="usd", days=90):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        "vs_currency": vs_currency,
        "days": days,
        "interval": "hourly",  # Fetch hourly data
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

# Function to process and filter data
def process_historical_data(data):
    prices = data["prices"]
    market_caps = data["market_caps"]
    total_volumes = data["total_volumes"]

    # Combine data into a single DataFrame
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["market_cap"] = [cap[1] for cap in market_caps]
    df["volume"] = [vol[1] for vol in total_volumes]

    # Convert timestamp to datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df["date"] = df["timestamp"].dt.date
    df["hour"] = df["timestamp"].dt.hour

    # Filter for 12 AM and 12 PM data points
    filtered_df = df[(df["hour"] == 0) | (df["hour"] == 12)]

    # Rename columns for clarity
    filtered_df.rename(
        columns={
            "price": "close",
            "market_cap": "market_high",
            "volume": "market_low",
        },
        inplace=True,
    )
    
    # Add open, high, low (mock data for demonstration)
    filtered_df["open"] = filtered_df["close"] * 0.98  # Example calculation
    filtered_df["high"] = filtered_df["close"] * 1.02
    filtered_df["low"] = filtered_df["close"] * 0.95

    return filtered_df

# Main function to fetch and display historical data
def main():
    coin_id = "bitcoin"  # Change to desired cryptocurrency ID
    days = 90
    data = fetch_historical_data(coin_id, days=days)
    
    if data:
        processed_data = process_historical_data(data)
        print(f"\nHistorical Data for {coin_id.capitalize()} (Past {days} Days):\n")
        print(processed_data.head(20))  # Display first 20 rows
    else:
        print("No data fetched.")

# Run the script
if __name__ == "__main__":
    main()


Error: 401
No data fetched.


In [ ]:
import requests
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pickle
from datetime import datetime, timedelta

# Constants
API_URL = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
MODEL_FILE = "incremental_price_model.pkl"

# Fetch historical data from CoinGecko API
def fetch_historical_data(days=30):
    params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "hourly"
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        prices = data["prices"]  # [timestamp, price]
        df = pd.DataFrame(prices, columns=["timestamp", "price"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        return df
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

# Prepare data for training
def prepare_data(df):
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    df["weekday"] = df["timestamp"].dt.weekday
    X = df[["hour", "day", "month", "weekday"]]
    y = df["price"]
    return X, y

# Load or initialize model
def load_or_initialize_model():
    try:
        with open(MODEL_FILE, "rb") as f:
            model = pickle.load(f)
        print("Model loaded.")
    except FileNotFoundError:
        model = make_pipeline(StandardScaler(), SGDRegressor())
        print("New model initialized.")
    return model

# Save model
def save_model(model):
    with open(MODEL_FILE, "wb") as f:
        pickle.dump(model, f)
    print("Model saved.")

# Predict current price
def predict_price(model):
    now = datetime.now()
    input_data = pd.DataFrame([{
        "hour": now.hour,
        "day": now.day,
        "month": now.month,
        "weekday": now.weekday()
    }])
    prediction = model.predict(input_data)
    return prediction[0]

# Main function
def main():
    print("Fetching historical data...")
    df = fetch_historical_data(days=1)  # Fetch only the last day's data for incremental training
    
    print("Preparing data...")
    X, y = prepare_data(df)
    
    print("Loading or initializing model...")
    model = load_or_initialize_model()
    
    print("Incremental training model...")
    if hasattr(model.named_steps['sgdregressor'], "partial_fit"):
        # SGDRegressor requires `partial_fit` with initial classes
        model.named_steps['sgdregressor'].partial_fit(X, y)
    else:
        print("Model does not support incremental learning.")
        return
    
    print("Saving model...")
    save_model(model)
    
    print("Predicting current price...")
    predicted_price = predict_price(model)
    print(f"Predicted current price: ${predicted_price:.2f}")

if __name__ == "__main__":
    main()


In [1]:
import pandas as pd

In [18]:
data = pd.read_csv("Binance_1INCHBTC_d.csv",skiprows=1)
data.describe()

,Unix,Open,High,Low,Close,Volume 1INCH,Volume BTC,tradecount
count,1.467000e+03,1467.000000,1467.000000,1467.000000,1467.000000,1.467000e+03,1467.000000,1467.000000
mean,1.672186e+12,0.000032,0.000034,0.000031,0.000032,9.486608e+05,53.667465,7700.021813
std,3.660170e+10,0.000029,0.000031,0.000027,0.000029,1.826472e+06,156.399689,14501.655486
min,1.608854e+12,0.000003,0.000003,0.000003,0.000003,1.328310e+04,0.046074,85.000000
25%,1.640520e+12,0.000009,0.000009,0.000009,0.000009,2.038166e+05,2.222557,1251.000000
50%,1.672186e+12,0.000025,0.000025,0.000024,0.000025,4.146247e+05,8.763403,3023.000000
75%,1.703851e+12,0.000050,0.000052,0.000048,0.000050,1.005952e+06,40.802332,7512.000000
max,1.735517e+12,0.000152,0.000168,0.000148,0.000152,3.191545e+07,3145.367039,226447.000000


In [25]:
display(data)

,Unix,Date,Symbol,Open,High,Low,Close,Volume 1INCH,Volume BTC,tradecount
0,1735516800000,2024-12-30,1INCHBTC,0.000004,0.000004,0.000004,0.000004,210753.3,0.886997,832
1,1735430400000,2024-12-29,1INCHBTC,0.000004,0.000004,0.000004,0.000004,77095.2,0.324914,327
2,1735344000000,2024-12-28,1INCHBTC,0.000004,0.000004,0.000004,0.000004,87745.4,0.371705,543
3,1735257600000,2024-12-27,1INCHBTC,0.000004,0.000004,0.000004,0.000004,90503.4,0.375294,419
4,1735171200000,2024-12-26,1INCHBTC,0.000004,0.000004,0.000004,0.000004,173449.7,0.712950,525
...,...,...,...,...,...,...,...,...,...,...
1462,1609200000000,2020-12-29,1INCHBTC,0.000041,0.000041,0.000029,0.000032,11666843.3,391.163430,48704
1463,1609113600000,2020-12-28,1INCHBTC,0.000040,0.000047,0.000040,0.000041,9797253.1,421.125515,42828
1464,1609027200000,2020-12-27,1INCHBTC,0.000060,0.000062,0.000039,0.000040,18382333.9,883.260231,83251
1465,1608940800000,2020-12-26,1INCHBTC,0.000093,0.000100,0.000060,0.000060,16588695.5,1215.782890,93597


In [22]:
data.dropna()

,Unix,Date,Symbol,Open,High,Low,Close,Volume 1INCH,Volume BTC,tradecount
0,1735516800000,2024-12-30,1INCHBTC,0.000004,0.000004,0.000004,0.000004,210753.3,0.886997,832
1,1735430400000,2024-12-29,1INCHBTC,0.000004,0.000004,0.000004,0.000004,77095.2,0.324914,327
2,1735344000000,2024-12-28,1INCHBTC,0.000004,0.000004,0.000004,0.000004,87745.4,0.371705,543
3,1735257600000,2024-12-27,1INCHBTC,0.000004,0.000004,0.000004,0.000004,90503.4,0.375294,419
4,1735171200000,2024-12-26,1INCHBTC,0.000004,0.000004,0.000004,0.000004,173449.7,0.712950,525
...,...,...,...,...,...,...,...,...,...,...
1462,1609200000000,2020-12-29,1INCHBTC,0.000041,0.000041,0.000029,0.000032,11666843.3,391.163430,48704
1463,1609113600000,2020-12-28,1INCHBTC,0.000040,0.000047,0.000040,0.000041,9797253.1,421.125515,42828
1464,1609027200000,2020-12-27,1INCHBTC,0.000060,0.000062,0.000039,0.000040,18382333.9,883.260231,83251
1465,1608940800000,2020-12-26,1INCHBTC,0.000093,0.000100,0.000060,0.000060,16588695.5,1215.782890,93597


In [20]:
data.columns

Index(['Unix', 'Date', 'Symbol', 'Open', 'High', 'Low', 'Close',
       'Volume 1INCH', 'Volume BTC', 'tradecount'],
      dtype='object')

In [24]:
data.dtypes

Unix                     int64
Date            datetime64[ns]
Symbol                  object
Open                   float64
High                   float64
Low                    float64
Close                  float64
Volume 1INCH           float64
Volume BTC             float64
tradecount               int64
dtype: object

In [23]:
data['Date'] = pd.to_datetime(data['Date'])

In [47]:
x = data[['Open','High','Low','Volume 1INCH','Volume BTC','tradecount']]
y = data['Close']

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
X_train , X_test, y_train , y_test = train_test_split(x,y ,test_size=0.4)

In [50]:
print(X_train.shape,
y_train.shape,
x_test.shape,
y_test.shape)

(880, 6) (880,) (587, 7) (587,)


In [68]:
import pandas as pd
import warnings
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,r2_score


# Suppress warnings
warnings.filterwarnings("ignore")

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Models supporting partial_fit for regression
models = {
    "SGDRegressor": SGDRegressor(max_iter=100, tol=1e-3),
    "MLPRegressor": MLPRegressor(max_iter=100, warm_start=True),  # Warm start enables incremental training
}

# Initialize results dictionary
results = {"Model": [], "MSE": [], "Percentage Change": [], "r2_score" : []}

# Train each model using partial_fit and evaluate
baseline_mse = None  # Placeholder for baseline MSE (first model)

for name, model in models.items():
    if hasattr(model, "partial_fit"):
        if name == "MLPRegressor":  # MLPRegressor does not need `classes` parameter
            for i in range(10):  # Incremental training with 10 iterations
                model.partial_fit(X_train, y_train)
        else:
            model.partial_fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Calculate Mean Squared Error (MSE)
        mse = mean_squared_error(y_test, y_pred)
        
        #Ca1culate r2_score
        r2 = r2_score(y_test, y_pred)
        # Calculate percentage change from the baseline MSE
        if baseline_mse is None:
            baseline_mse = mse
            percentage_change = 0.0  # No change for the first model
        else:
            percentage_change = ((mse - baseline_mse) / baseline_mse) * 100
        
        # Store results with MSE rounded to 6 decimals
        results["Model"].append(name)
        results["MSE"].append(round(mse, 9))
        results["r2_score"].append(round(r2,1))

        results["Percentage Change"].append(round(percentage_change, 2))

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display results
print(results_df)


          Model       MSE  Percentage Change   r2_score
0  SGDRegressor  0.000000       0.000000e+00        1.0
1  MLPRegressor  0.001536       8.647142e+09 -2110229.2


In [11]:
import requests

# List of cryptocurrencies with their IDs on CoinGecko
currencies = {
    "Ether": "ethereum",
    "Akamaru": "akamaru",
    #"Carmine Token": "carmine-token",
    "Dai Stablecoin": "dai",
    "Dai Stablecoin (old)": "dai",
    "Ekubo Protocol": "ekubo-protocol",
    "LORDS": "lords",
    "LUSD": "liquity-usd",
    "Nostra": "nostra",
    "OWL": "owl",
    #"Pain au Lait": "pain-au-lait",
    "Paper": "paper",
    "Rocket Pool ETH": "rocket-pool-eth",
    #"Stark Pepe": "stark-pepe",
    "Starknet": "starknet",
    #"Standard Weighted Adalian Yield": "standard-weighted-adalian-yield",
    "USDCoin": "usd-coin",
    "Tether USD": "tether",
    "Wrapped BTC": "wrapped-bitcoin",
    "Wrapped stETH": "wrapped-steth"
}

def get_current_prices(currencies, currency="usd"):
    url = "https://api.coingecko.com/api/v3/simple/price"
    ids = ",".join(currencies.values())  # Join all CoinGecko IDs into a single string
    response = requests.get(url, params={"ids": ids, "vs_currencies": currency})
    
    if response.status_code == 200:
        prices = response.json()
        return {name: prices.get(coin_id, {}).get(currency, "Price not available") 
                for name, coin_id in currencies.items()}
    else:
        print(f"Failed to fetch prices: {response.status_code}")

# Fetch and print current prices
try:
    prices = get_current_prices(currencies)
    for name, price in prices.items():
        print(f"{name}: {price} USD")
except Exception as e:
    print(e)


Ether: 3413.12 USD
Akamaru: 9.358e-09 USD
Dai Stablecoin: 1.0 USD
Dai Stablecoin (old): 1.0 USD
Ekubo Protocol: 3.87 USD
LORDS: 0.152565 USD
LUSD: 1.001 USD
Nostra: 0.093078 USD
OWL: 0.27979 USD
Paper: 0.774262 USD
Rocket Pool ETH: 3826.07 USD
Starknet: 0.498803 USD
USDCoin: 1.001 USD
Tether USD: 0.998794 USD
Wrapped BTC: 95293 USD
Wrapped stETH: 4051.93 USD


In [7]:
display(currency_data)

{'Ether':               timestamp        price
 0   2024-09-25 00:00:00  2653.844345
 1   2024-09-26 00:00:00  2578.566360
 2   2024-09-27 00:00:00  2630.949837
 3   2024-09-28 00:00:00  2698.192821
 4   2024-09-29 00:00:00  2680.218702
 ..                  ...          ...
 96  2024-12-30 00:00:00  3357.330743
 97  2024-12-31 00:00:00  3359.513942
 98  2025-01-01 00:00:00  3336.617514
 99  2025-01-02 00:00:00  3348.967247
 100 2025-01-02 06:30:17  3400.605349
 
 [101 rows x 2 columns],
 'Akamaru':               timestamp         price
 0   2024-09-25 00:00:00  7.422286e-09
 1   2024-09-26 00:00:00  7.218236e-09
 2   2024-09-27 00:00:00  8.062653e-09
 3   2024-09-28 00:00:00  8.303683e-09
 4   2024-09-29 00:00:00  8.199347e-09
 ..                  ...           ...
 96  2024-12-30 00:00:00  9.145175e-09
 97  2024-12-31 00:00:00  9.166422e-09
 98  2025-01-01 00:00:00  9.105339e-09
 99  2025-01-02 00:00:00  9.217004e-09
 100 2025-01-02 06:28:43  9.349707e-09
 
 [101 rows x 2 columns],
 '

Get Data of past 30 days and partial fit the models

In [5]:
import requests
import pandas as pd
from datetime import datetime

API_URL = "https://api.coingecko.com/api/v3/coins/{id}/market_chart"

def fetch_last_30_days_data(coin_id="bitcoin", vs_currency="usd"):
    try:
        # Parameters for the API request
        params = {
            "vs_currency": vs_currency,
            "days": 30,  # Fetch data for the last 30 days
            "interval": "daily"  # Get daily data
        }

        # Format the URL with the coin ID
        url = API_URL.format(id=coin_id)

        # Send the GET request
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the JSON response
        data = response.json()

        # Extract required fields
        prices = data.get("prices", [])
        market_caps = data.get("market_caps", [])
        total_volumes = data.get("total_volumes", [])

        # Combine data into a list of dictionaries
        result = []
        for i in range(len(prices)):
            result.append({
                "snapped_at": datetime.utcfromtimestamp(prices[i][0] / 1000).strftime('%Y-%m-%d %H:%M:%S'),
                "price": prices[i][1],
                "market_cap": market_caps[i][1] if i < len(market_caps) else None,
                "total_volume": total_volumes[i][1] if i < len(total_volumes) else None,
            })

        # Convert the result to a Pandas DataFrame
        df = pd.DataFrame(result)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

# Example usage
if __name__ == "__main__":
    coin_id = "ethereum"  # Replace with the desired coin ID
    df = fetch_last_30_days_data(coin_id, vs_currency="usd")
    print(f"Fetched data for {coin_id}:")
    print(df.head())  # Display the first few rows of the DataFrame


Fetched data for ethereum:
            snapped_at        price    market_cap  total_volume
0  2024-12-07 00:00:00  4013.726145  4.834225e+11  5.595447e+10
1  2024-12-08 00:00:00  4000.992077  4.818892e+11  2.195979e+10
2  2024-12-09 00:00:00  4015.782021  4.836429e+11  2.072815e+10
3  2024-12-10 00:00:00  3713.313898  4.465713e+11  5.914140e+10
4  2024-12-11 00:00:00  3626.588642  4.367496e+11  6.364351e+10


In [6]:
display(df)

,snapped_at,price,market_cap,total_volume
0,2024-12-07 00:00:00,4013.726145,4.834225e+11,5.595447e+10
1,2024-12-08 00:00:00,4000.992077,4.818892e+11,2.195979e+10
2,2024-12-09 00:00:00,4015.782021,4.836429e+11,2.072815e+10
3,2024-12-10 00:00:00,3713.313898,4.465713e+11,5.914140e+10
4,2024-12-11 00:00:00,3626.588642,4.367496e+11,6.364351e+10
5,2024-12-12 00:00:00,3828.107588,4.610759e+11,3.847162e+10
6,2024-12-13 00:00:00,3878.850012,4.668831e+11,4.747461e+10
7,2024-12-14 00:00:00,3907.429660,4.704699e+11,3.621832e+10
8,2024-12-15 00:00:00,3866.995802,4.657133e+11,2.864772e+10
9,2024-12-16 00:00:00,3961.315466,4.773232e+11,2.481685e+10
